In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wget as wget

Dataset

In [4]:
wget.download("https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv")

'course_lead_scoring.csv'

Data preparation

In [5]:
df = pd.read_csv("course_lead_scoring.csv")
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [6]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [7]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [8]:
numerical_vars = df.select_dtypes(include=['int64', 'float64'])
categorical_vars = df.select_dtypes(include=['object'])
 
print("Numerical Variables:")
print(numerical_vars.columns)
 
print("\nCategorical Variables:")
print(categorical_vars.columns)

Numerical Variables:
Index(['number_of_courses_viewed', 'annual_income', 'interaction_count',
       'lead_score', 'converted'],
      dtype='object')

Categorical Variables:
Index(['lead_source', 'industry', 'employment_status', 'location'], dtype='object')


In [19]:
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count','lead_score']

categorical = ['lead_source', 'industry', 'employment_status', 'location']

In [20]:
# Replace missing values in categorical features with 'NA'
df[categorical] = df[categorical].fillna('NA')

# Replace missing values in numerical features with 0.0
df[numerical] = df[numerical].fillna(0.0)

In [21]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

Question 1

In [22]:
df['industry'].mode()[0]

'retail'

Question 2

In [23]:
# Compute the correlation matrix for numerical features
corr_matrix = df[numerical].corr()

# Display the correlation matrix and the two most correlated features
print("Correlation Matrix:\n", corr_matrix)


Correlation Matrix:
                           number_of_courses_viewed  annual_income  \
number_of_courses_viewed                  1.000000       0.009770   
annual_income                             0.009770       1.000000   
interaction_count                        -0.023565       0.027036   
lead_score                               -0.004879       0.015610   

                          interaction_count  lead_score  
number_of_courses_viewed          -0.023565   -0.004879  
annual_income                      0.027036    0.015610  
interaction_count                  1.000000    0.009888  
lead_score                         0.009888    1.000000  


In [24]:
# Find the pair of numerical features with the highest absolute correlation (excluding self-correlation)
corr_matrix_no_diag = corr_matrix.where(~np.eye(corr_matrix.shape[0], dtype=bool))
max_corr = corr_matrix_no_diag.abs().unstack().sort_values(ascending=False).drop_duplicates()
top_pair = max_corr.index[0]
print(f"The two numerical features with the biggest correlation are: {top_pair[0]} and {top_pair[1]}")

The two numerical features with the biggest correlation are: annual_income and interaction_count


In [25]:
from sklearn.model_selection import train_test_split

In [39]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
len(df_full_train), len(df_test)

(1169, 293)

In [40]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [41]:
df_full_train = df_full_train.reset_index(drop=True)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [42]:
y_full_train = df_full_train.converted.values
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

In [43]:
#del df_full_train['converted']
del df_train['converted']
del df_val['converted']
del df_test['converted']

Question 3

In [44]:
from sklearn.metrics import mutual_info_score

In [45]:
def mutual_info_converted_score(series):
    return mutual_info_score(series, df_full_train.converted)

mi = df_full_train[categorical].apply(mutual_info_converted_score)
mi.sort_values(ascending=False)

lead_source          0.025665
employment_status    0.013258
industry             0.011685
location             0.002253
dtype: float64

Question 4

In [46]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [47]:
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
 
dv.fit(train_dicts)
X_train = dv.transform(train_dicts)

In [48]:
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [49]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [50]:
y_pred = model.predict_proba(X_val)[:,1]

convert_decision = (y_pred >= 0.5)
(convert_decision == y_val).mean()

0.7440273037542662

Question 5

In [51]:
# Original feature set and accuracy
features = categorical + numerical
train_dicts = df_train[features].to_dict(orient='records')
val_dicts = df_val[features].to_dict(orient='records')
X_train = dv.transform(train_dicts)
X_val = dv.transform(val_dicts)

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_val)[:, 1]
original_acc = ((y_pred >= 0.5) == y_val).mean()

# Feature elimination
results = {}
for feature in features:
    reduced_features = [f for f in features if f != feature]
    train_dicts_reduced = df_train[reduced_features].to_dict(orient='records')
    val_dicts_reduced = df_val[reduced_features].to_dict(orient='records')
    X_train_reduced = dv.transform(train_dicts_reduced)
    X_val_reduced = dv.transform(val_dicts_reduced)
    
    model_reduced = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model_reduced.fit(X_train_reduced, y_train)
    y_pred_reduced = model_reduced.predict_proba(X_val_reduced)[:, 1]
    acc_reduced = ((y_pred_reduced >= 0.5) == y_val).mean()
    
    results[feature] = original_acc - acc_reduced

results

{'lead_source': 0.013651877133105783,
 'industry': 0.010238907849829282,
 'employment_status': 0.010238907849829282,
 'location': 0.0,
 'number_of_courses_viewed': 0.03412969283276446,
 'annual_income': -0.13993174061433455,
 'interaction_count': 0.03754266211604096,
 'lead_score': 0.0034129692832763903}

Question 5

In [55]:
C_values = [0.01, 0.1, 1, 10, 100]
accuracies = []

for C in C_values:
    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_val)[:, 1]
    acc = ((y_pred >= 0.5) == y_val).mean()
    accuracies.append(round(acc, 3))

dict(zip(C_values, accuracies))

{0.01: 0.744, 0.1: 0.744, 1: 0.744, 10: 0.744, 100: 0.744}